# 👤 Analysis 4 — User Crossposting Behaviour

**Core question:** Which users post across ideologically opposed subreddits? What does that overlap look like?

**Method:**  
1. For every pair of subreddits, count authors who posted in BOTH  
2. Compute Jaccard similarity: `|A ∩ B| / |A ∪ B|`  
3. Build a subreddit similarity matrix based on shared audience  

**Interesting pairs to find:**  
- r/politics ↔ r/conservative (do they share users or are they completely separate camps?)  
- r/wallstreetbets ↔ r/collapse (doom + stonks overlap?)  
- r/femaledatingstrategy ↔ r/dating_advice  

**Interview talking point:**  
> "I used a self-join on the author column to find cross-community users, then computed Jaccard similarity to build a subreddit audience overlap matrix. This is the same pattern used in recommendation systems — if two communities share 30% of their audience, they're likely to surface each other's content."


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = (
    SparkSession.builder.appName('Crosspost')
    .master('local[2]')
    .config('spark.driver.memory', '3g')
    .config('spark.sql.shuffle.partitions', '8')
    .getOrCreate()
)
spark.sparkContext.setLogLevel('WARN')

df = spark.read.parquet('/mnt/c/Users/gusmc/OneDrive/Desktop/reddit_historical_data/data/silver/posts')

# Filter out bots and deleted accounts
df = df.filter(
    F.col('author').isNotNull() &
    ~F.col('author').isin('[deleted]', 'automoderator', 'bot') &
    ~F.col('author').endswith('bot')
)

print(f'Posts: {df.count():,}')

26/02/27 04:41:26 WARN Utils: Your hostname, terminator resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/27 04:41:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/27 04:41:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[Stage 1:=======================================================> (44 + 1) / 45]

Posts: 8,301,494


In [2]:
# ── 1. Distinct authors per subreddit ─────────────────────────────────────────
author_sub = (
    df.select('author', 'subreddit')
    .distinct()   # one row per (author, subreddit) pair
)

sub_sizes = (
    author_sub.groupBy('subreddit')
    .agg(F.count('author').alias('unique_authors'))
    .orderBy(F.desc('unique_authors'))
)

print('=== UNIQUE AUTHORS PER SUBREDDIT ===')
sub_sizes.show(20, truncate=False)

=== UNIQUE AUTHORS PER SUBREDDIT ===


+--------------------+--------------+
|subreddit           |unique_authors|
+--------------------+--------------+
|worldnews           |396450        |
|politics            |286722        |
|trueoffmychest      |251634        |
|antiwork            |235331        |
|wallstreetbets      |182515        |
|unpopularopinion    |182049        |
|aitah               |166406        |
|conservative        |88498         |
|changemyview        |76580         |
|formuladank         |72650         |
|soccercirclejerk    |57712         |
|dating_advice       |38382         |
|collapse            |20987         |
|femaledatingstrategy|12222         |
+--------------------+--------------+



In [3]:
# ── 2. Cross-subreddit overlap — self join ────────────────────────────────────
# Self-join: for every (author, sub_A) row, join with all (author, sub_B) rows
# This finds all users who post in at least 2 different subreddits

# Alias the same dataframe twice
a = author_sub.alias('a')
b = author_sub.alias('b')

# Join on same author, different subreddits
# a.subreddit < b.subreddit avoids counting (politics,worldnews) AND (worldnews,politics)
pairs = (
    a.join(b,
        (F.col('a.author') == F.col('b.author')) &
        (F.col('a.subreddit') < F.col('b.subreddit')),
        how='inner'
    )
    .select(
        F.col('a.subreddit').alias('sub_a'),
        F.col('b.subreddit').alias('sub_b'),
        F.col('a.author').alias('author')
    )
)

# Count shared authors per pair
overlap = (
    pairs.groupBy('sub_a', 'sub_b')
    .agg(F.count('author').alias('shared_authors'))
)

print('=== RAW OVERLAP (TOP 20 PAIRS BY SHARED AUTHORS) ===')
overlap.orderBy(F.desc('shared_authors')).show(20, truncate=False)

=== RAW OVERLAP (TOP 20 PAIRS BY SHARED AUTHORS) ===


[Stage 15:>                                                         (0 + 2) / 2]

+----------------+----------------+--------------+
|sub_a           |sub_b           |shared_authors|
+----------------+----------------+--------------+
|politics        |worldnews       |52887         |
|trueoffmychest  |unpopularopinion|13671         |
|antiwork        |unpopularopinion|9675          |
|antiwork        |trueoffmychest  |8922          |
|changemyview    |unpopularopinion|4913          |
|wallstreetbets  |worldnews       |4460          |
|unpopularopinion|wallstreetbets  |4182          |
|politics        |wallstreetbets  |3852          |
|antiwork        |wallstreetbets  |3849          |
|changemyview    |politics        |3846          |
|antiwork        |conservative    |3657          |
|conservative    |unpopularopinion|3578          |
|antiwork        |collapse        |3398          |
|conservative    |worldnews       |3222          |
|changemyview    |worldnews       |3025          |
|changemyview    |trueoffmychest  |2897          |
|conservative    |wallstreetbet

In [4]:
# ── 3. Jaccard similarity ─────────────────────────────────────────────────────
# Jaccard = |A ∩ B| / |A ∪ B|
# Normalises for community size — raw overlap favours large subs

sizes_a = sub_sizes.select(
    F.col('subreddit').alias('sub_a'),
    F.col('unique_authors').alias('size_a')
)
sizes_b = sub_sizes.select(
    F.col('subreddit').alias('sub_b'),
    F.col('unique_authors').alias('size_b')
)

jaccard = (
    overlap
    .join(sizes_a, on='sub_a')
    .join(sizes_b, on='sub_b')
    .withColumn('union_size',
        F.col('size_a') + F.col('size_b') - F.col('shared_authors')
    )
    .withColumn('jaccard_similarity',
        F.round(
            F.col('shared_authors').cast('double') / F.col('union_size'),
            4
        )
    )
    .select('sub_a', 'sub_b', 'shared_authors', 'size_a', 'size_b', 'jaccard_similarity')
    .orderBy(F.desc('jaccard_similarity'))
)

print('=== JACCARD SIMILARITY (most similar audiences) ===')
jaccard.show(30, truncate=False)

=== JACCARD SIMILARITY (most similar audiences) ===


+----------------+----------------+--------------+------+------+------------------+
|sub_a           |sub_b           |shared_authors|size_a|size_b|jaccard_similarity|
+----------------+----------------+--------------+------+------+------------------+
|politics        |worldnews       |52887         |286722|396450|0.0839            |
|trueoffmychest  |unpopularopinion|13671         |251634|182049|0.0325            |
|antiwork        |unpopularopinion|9675          |235331|182049|0.0237            |
|changemyview    |unpopularopinion|4913          |76580 |182049|0.0194            |
|antiwork        |trueoffmychest  |8922          |235331|251634|0.0187            |
|formuladank     |soccercirclejerk|1941          |72650 |57712 |0.0151            |
|conservative    |unpopularopinion|3578          |88498 |182049|0.0134            |
|antiwork        |collapse        |3398          |235331|20987 |0.0134            |
|unpopularopinion|wallstreetbets  |4182          |182049|182515|0.0116      

In [5]:
# ── 4. Multi-community users — who posts everywhere? ─────────────────────────
# These are the most cross-cutting users in your dataset

user_breadth = (
    author_sub
    .groupBy('author')
    .agg(
        F.count('subreddit').alias('sub_count'),
        F.collect_set('subreddit').alias('subreddits')
    )
    .filter(F.col('sub_count') >= 3)  # posts in at least 3 of your subs
    .orderBy(F.desc('sub_count'))
)

print('=== USERS ACTIVE IN 3+ SUBREDDITS ===')
print(f'Count: {user_breadth.count():,}')
user_breadth.show(20, truncate=False)

=== USERS ACTIVE IN 3+ SUBREDDITS ===


Count: 15,285


[Stage 47:>                                                         (0 + 2) / 2]

+-------------------+---------+------------------------------------------------------------------------------------------------------------------+
|author             |sub_count|subreddits                                                                                                        |
+-------------------+---------+------------------------------------------------------------------------------------------------------------------+
|123456american     |8        |[unpopularopinion, worldnews, changemyview, collapse, politics, wallstreetbets, conservative, antiwork]           |
|bman409            |8        |[antiwork, worldnews, changemyview, politics, collapse, wallstreetbets, conservative, unpopularopinion]           |
|kinglens311        |8        |[soccercirclejerk, worldnews, collapse, formuladank, wallstreetbets, femaledatingstrategy, conservative, antiwork]|
|buhdumtssss        |7        |[unpopularopinion, worldnews, collapse, dating_advice, trueoffmychest, conservative, an

In [6]:
# ── 5. Ideological crossover — specific interesting pairs ─────────────────────
interesting_pairs = [
    ('politics', 'conservative'),
    ('wallstreetbets', 'collapse'),
    ('femaledatingstrategy', 'dating_advice'),
    ('worldnews', 'collapse'),
    ('changemyview', 'unpopularopinion'),
]

print('=== SPECIFIC IDEOLOGICAL CROSSOVERS ===')
for sub_a, sub_b in interesting_pairs:
    # Normalise order
    a_, b_ = sorted([sub_a, sub_b])
    result = jaccard.filter(
        (F.col('sub_a') == a_) & (F.col('sub_b') == b_)
    ).collect()
    if result:
        r = result[0]
        print(f'{sub_a:30s} ↔ {sub_b:30s}  shared={r.shared_authors:,}  jaccard={r.jaccard_similarity}')
    else:
        print(f'{sub_a} ↔ {sub_b}  — no overlap found')

=== SPECIFIC IDEOLOGICAL CROSSOVERS ===


politics                       ↔ conservative                    shared=1,538  jaccard=0.0041


wallstreetbets                 ↔ collapse                        shared=829  jaccard=0.0041
femaledatingstrategy           ↔ dating_advice                   shared=209  jaccard=0.0041


worldnews                      ↔ collapse                        shared=1,440  jaccard=0.0035


changemyview                   ↔ unpopularopinion                shared=4,913  jaccard=0.0194


In [7]:
spark.stop()
print('Crossposting analysis complete ✓')

Crossposting analysis complete ✓
